# ML model Training (Question 3 of the Homework)

In [42]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
# Read datasets

df_pe = pd.read_csv("../data/pe/cleaned/pe_dataset.csv")
normalized_df_pe = pd.read_csv("../data/pe/cleaned/normalized_pe_dataset.csv")
normalized_df_pe_without_outliers = pd.read_csv("../data/pe/cleaned/normalized_pe_dataset_without_outliers.csv")

In [18]:
def splitDataset(df, test_size=0.15, should_have_validation_set=True):
    from sklearn.model_selection import train_test_split
    X = df.drop(columns=['Entity', 'Year','Primary energy consumption per capita (kWh/person)'])
    y = df['Primary energy consumption per capita (kWh/person)']
    X_pretrain, X_test, y_pretrain, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=None)

    if should_have_validation_set:
        X_train, X_val, y_train, y_val = train_test_split(X_pretrain, y_pretrain, test_size=test_size/(1-test_size), shuffle=True, random_state=None)
        print(X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape)
        return X_train, X_test, X_val, y_train, y_test, y_val
    
    print(X_pretrain.shape, X_test.shape, y_pretrain.shape, y_test.shape)
    return X_pretrain, X_test, y_pretrain, y_test

## Linear Regression

As seen in the [ML algorithm selection notebook](ML_algorithm_selection.ipynb), the linear model does not have any hyperparameters to tune. Therefore, cross-validation is not required. We can split our datasets into training, testing and validation sets like so:

- Training set: 80%
- Testing set: 20%

To do that, I'll use the `train_test_split` function from `sklearn.model_selection` module. This function provides parameters to shuffle or not our dataset before splitting it. I'll set the `shuffle` parameter to `True` to ensure that the data is shuffled before splitting it. According to [scikit-learn documentation](https://scikit-learn.org/stable/glossary.html#term-random_state), by default the function will use the global random state from `np.random` module to shuffle the data. 

NOTE: I don't see the use here of having reproducible results, so I'll leave the `random_state` parameter to `None`. 

In [21]:
# Split datasets into train, test and validation sets
X_pe_train, X_pe_test, y_pe_train, y_pe_test = splitDataset(df_pe, test_size=0.2, should_have_validation_set=False)

# nor -> normalized
X_nor_pe_train, X_nor_pe_test, y_nor_pe_train, y_nor_pe_test = splitDataset(normalized_df_pe, test_size=0.2, should_have_validation_set=False)

# nor_wo -> normalized without outliers
X_nor_pe_wo_train, X_nor_pe_wo_test, y_nor_pe_wo_train, y_nor_pe_wo_test = splitDataset(normalized_df_pe_without_outliers, test_size=0.2, should_have_validation_set=False)


(2554, 4) (639, 4) (2554,) (639,)
(2554, 4) (639, 4) (2554,) (639,)
(2407, 4) (602, 4) (2407,) (602,)


Once we have our subsets, we can train our model using the training set for each dataset.

In [37]:
# Model training

from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()
LR_model_nor = LinearRegression()
LR_model_nor_wo = LinearRegression()

In [38]:
# Train on global dataset

LR_model.fit(X_pe_train, y_pe_train)
print(f'R^2 score: {LR_model.score(X_pe_train, y_pe_train).round(3)}')
LR_coef = pd.DataFrame(LR_model.coef_, index=X_pe_train.columns, columns=['Coefficient'])
LR_coef.loc['Intercept'] = LR_model.intercept_
LR_coef.round(3)

R^2 score: 0.55


,Coefficient
Access to electricity (% of population),-86.645
Access to clean fuels for cooking,183.810
Renewable energy share in the total final energy consumption (%),-164.559
gdp_per_capita,1.159
Intercept,12381.889


In [39]:
# Train on normalized dataset

LR_model_nor.fit(X_nor_pe_train, y_nor_pe_train)
print(f'R^2 score: {LR_model_nor.score(X_nor_pe_train, y_nor_pe_train).round(3)}')
LR_coef_nor = pd.DataFrame(LR_model_nor.coef_, index=X_nor_pe_train.columns, columns=['Coefficient'])
LR_coef_nor.loc['Intercept'] = LR_model_nor.intercept_
LR_coef_nor.round(3)

R^2 score: 0.545


,Coefficient
Access to electricity (% of population),-0.030
Access to clean fuels for cooking,0.074
Renewable energy share in the total final energy consumption (%),-0.049
gdp_per_capita,0.569
Intercept,0.038


In [40]:
# Train on normalized dataset without outliers

LR_model_nor_wo.fit(X_nor_pe_wo_train, y_nor_pe_wo_train)
print(f'R^2 score: {LR_model_nor_wo.score(X_nor_pe_wo_train, y_nor_pe_wo_train).round(3)}')
LR_coef_nor_wo = pd.DataFrame(LR_model_nor_wo.coef_, index=X_nor_pe_wo_train.columns, columns=['Coefficient'])
LR_coef_nor_wo.loc['Intercept'] = LR_model_nor_wo.intercept_
LR_coef_nor_wo.round(3)

R^2 score: 0.707


,Coefficient
Access to electricity (% of population),-0.031
Access to clean fuels for cooking,0.175
Renewable energy share in the total final energy consumption (%),-0.077
gdp_per_capita,0.816
Intercept,0.056


Steps with metrics to evaluate our model: https://www.freecodecamp.org/news/how-to-build-and-train-linear-and-logistic-regression-ml-models-in-python/

## Elastic Net

As seen in the [ML algorithm selection notebook](ML_algorithm_selection.ipynb), the Elastic Net model do have 2 hyperparameters to tune. Therefore, we can use cross-validation to find the best combination of hyperparameters. The following procedure is followed for each of the k folds:

- A model is trained using k-1 of the folds as training data;
- The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

![k-fold cross-validation](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)


The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop

The scikit-learn library provides a built-in version of the Elastic Net algorithm that automatically finds good hyperparameters using cross-validation: [ElasticNetCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html#sklearn.linear_model.ElasticNetCV).

So, to use this model, we need to set:

- An array of $\alpha$ values to test;
- An array of $l1_{ratio}$ values to test; According to scikit-learn documentation on this model:

    > A good choice of list of values for l1_ratio is often to put more values close to 1 (i.e. Lasso) and less close to 0 (i.e. Ridge), as in [.1, .5, .7, .9, .95, .99, 1].
- The number of folds to use for cross-validation. (k = 10 is a good default value).

To develop the model with K-Fold Cross Validation: https://machinelearningmastery.com/elastic-net-regression-in-python/

In [74]:
# Model training

from sklearn.linear_model import ElasticNetCV

l1_ratios = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0]

EN_model = ElasticNetCV(l1_ratio=l1_ratios, alphas=alphas, cv=10)
EN_model_nor = ElasticNetCV(l1_ratio=l1_ratios, alphas=alphas, cv=10)
EN_model_nor_wo = ElasticNetCV(l1_ratio=l1_ratios, alphas=alphas, cv=10)

In [80]:
# Train on global dataset

X_pe_train, X_pe_test, y_pe_train, y_pe_test = splitDataset(df_pe, test_size=0.2, should_have_validation_set=False)

EN_model.fit(X_pe_train, y_pe_train)

print(f'alpha: {EN_model.alpha_}')
print(f'l1_ratio: {EN_model.l1_ratio_}')
print(f'R^2 score: {EN_model.score(X_pe_train, y_pe_train).round(3)}')

EN_model_coef = pd.DataFrame(EN_model.coef_, index=X_pe_train.columns, columns=['Coefficient'])
EN_model_coef.loc['Intercept'] = EN_model.intercept_
EN_model_coef.round(3)

(2554, 4) (639, 4) (2554,) (639,)
alpha: 10.0
l1_ratio: 0.5
R^2 score: 0.546


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3455134534.0515137, tolerance: 283195722.6827962
  model = cd_fast.enet_coordinate_descent_gram(


,Coefficient
Access to electricity (% of population),-76.970
Access to clean fuels for cooking,196.062
Renewable energy share in the total final energy consumption (%),-141.634
gdp_per_capita,1.148
Intercept,10334.915


In [81]:
# Train on normalized dataset

X_nor_pe_train, X_nor_pe_test, y_nor_pe_train, y_nor_pe_test = splitDataset(normalized_df_pe, test_size=0.2, should_have_validation_set=False)

EN_model_nor.fit(X_nor_pe_train, y_nor_pe_train)

print(f'alpha: {EN_model_nor.alpha_}')
print(f'l1_ratio: {EN_model_nor.l1_ratio_}')
print(f'R^2 score: {EN_model_nor.score(X_nor_pe_train, y_nor_pe_train).round(3)}')

EN_model_nor_coef = pd.DataFrame(EN_model_nor.coef_, index=X_nor_pe_train.columns, columns=['Coefficient'])
EN_model_nor_coef.loc['Intercept'] = EN_model_nor.intercept_
EN_model_nor_coef.round(3)

(2554, 4) (639, 4) (2554,) (639,)
alpha: 1e-05
l1_ratio: 0.5
R^2 score: 0.544


,Coefficient
Access to electricity (% of population),-0.030
Access to clean fuels for cooking,0.076
Renewable energy share in the total final energy consumption (%),-0.050
gdp_per_capita,0.540
Intercept,0.039


In [82]:
# Train on normalized dataset without outliers

X_nor_pe_wo_train, X_nor_pe_wo_test, y_nor_pe_wo_train, y_nor_pe_wo_test = splitDataset(normalized_df_pe_without_outliers, test_size=0.2, should_have_validation_set=False)

EN_model_nor_wo.fit(X_nor_pe_wo_train, y_nor_pe_wo_train)

print(f'alpha: {EN_model_nor_wo.alpha_}')
print(f'l1_ratio: {EN_model_nor_wo.l1_ratio_}')
print(f'R^2 score: {EN_model_nor_wo.score(X_nor_pe_wo_train, y_nor_pe_wo_train).round(3)}')

EN_model_nor_wo_coef = pd.DataFrame(EN_model_nor_wo.coef_, index=X_nor_pe_wo_train.columns, columns=['Coefficient'])
EN_model_nor_wo_coef.loc['Intercept'] = EN_model_nor_wo.intercept_
EN_model_nor_wo_coef.round(3)

(2407, 4) (602, 4) (2407,) (602,)
alpha: 1e-05
l1_ratio: 0.1
R^2 score: 0.716


,Coefficient
Access to electricity (% of population),-0.036
Access to clean fuels for cooking,0.166
Renewable energy share in the total final energy consumption (%),-0.095
gdp_per_capita,0.875
Intercept,0.071
